In [87]:
# Header from RIT notebook
import pandas as pd
import matplotlib.pyplot as plt
import shutil
%matplotlib inline
##Setting up Google sdk environment
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jhlu/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-nigam-starr' #som-nero-phi-rit
%load_ext google.cloud.bigquery
##Setting up BQ API
from google.cloud import bigquery
client=bigquery.Client()
import numpy as np
import pandas as pd
import datetime
import collections
from IPython.display import display, HTML
import json

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/home/jhlu/.local/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [88]:
# just some formatting for figure making
plt.rcParams['figure.figsize'] = [12.0, 8.0]
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['figure.titlesize'] = 18
plt.rcParams['font.size'] = 18

pd.options.display.max_columns = 100

In [89]:
metadata = collections.OrderedDict()

metadata["Corresponding email addresses"] = "marcelkc@stanford.edu, jhlu@stanford.edu"
print("Corresponding email addresses: ", metadata["Corresponding email addresses"])

metadata["Code repository"] = "https://github.com/lujonathanh/Stanford_Starr-OMOP_COVID_MMWR"
print("Code repository: ", metadata["Code repository"])

run_date = datetime.datetime.today().strftime("%Y-%m-%d")
metadata["Run date"] = run_date
print("Run date:", run_date)

ARI_start_date = '2020-01-01'
metadata["ARI_start_date"] = ARI_start_date
print("ARI start date: ", ARI_start_date)

sarscov2test_concept_id = 706170
metadata["sarscov2test_concept_id"] = sarscov2test_concept_id
print("sarscov2test_concept_id:", sarscov2test_concept_id)



Corresponding email addresses:  marcelkc@stanford.edu, jhlu@stanford.edu
Code repository:  https://github.com/lujonathanh/Stanford_Starr-OMOP_COVID_MMWR
Run date: 2020-06-06
ARI start date:  2020-01-01
sarscov2test_concept_id: 706170


# Pull data

In [90]:
query = """
WITH

-- all visits involving an acute respiratory illness ICD-10 diagnosis with date since ARI_start_date
recentARI AS
( 
SELECT
    condOccur.person_id, visit_occurrence_id, condition_start_DATE, condition_start_DATETIME, 
    condition_source_value, condition_source_concept_id,
    visit_concept_id, care_site_name
  FROM
    `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.condition_occurrence` AS condOccur 
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE
    condition_start_DATE >= """ + "'" +  ARI_start_date + "'" + """ AND
    ( condition_source_value LIKE 'J00.%' OR
      condition_source_value LIKE 'J01.%' OR
      condition_source_value LIKE 'J02.%' OR
      condition_source_value LIKE 'J03.%' OR
      condition_source_value LIKE 'J04.%' OR
      condition_source_value LIKE 'J05.%' OR
      condition_source_value LIKE 'J06.%' OR
      condition_source_value LIKE 'J09.%' OR
      condition_source_value LIKE 'J10.%' OR
      condition_source_value LIKE 'J11.%' OR
      condition_source_value LIKE 'J12.%' OR
      condition_source_value LIKE 'J13.%' OR
      condition_source_value LIKE 'J14.%' OR
      condition_source_value LIKE 'J15.%' OR
      condition_source_value LIKE 'J16.%' OR
      condition_source_value LIKE 'J17.%' OR
      condition_source_value LIKE 'J18.%' OR
      condition_source_value LIKE 'J20.%' OR
      condition_source_value LIKE 'J21.%' OR
      condition_source_value LIKE 'J22' OR
      condition_source_value LIKE 'J80' OR
      condition_source_value LIKE 'A37.91' OR
      condition_source_value LIKE 'A37.01' OR
      condition_source_value LIKE 'A37.11' OR
      condition_source_value LIKE 'A37.81' OR
      condition_source_value LIKE 'A48.1' OR
      condition_source_value LIKE 'B25.0' OR
      condition_source_value LIKE 'B44.0' OR
      condition_source_value LIKE 'B97.4' OR
      condition_source_value LIKE 'U07.1'
    )
),


-- patient's first ARI visit in date range
firstARIInfo AS
(
SELECT person_id, firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS firstARIDateTime,
    visit_concept_id AS firstARI_visit_concept_id,
    care_site_name AS firstARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME ASC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),


-- patient's last ARI visit in date range
lastARIInfo AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS lastARIDateTime,
    visit_concept_id AS lastARI_visit_concept_id,
    care_site_name AS lastARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME DESC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),


-- Inner join patient's first and last ARI visits
recentARIDateTimeRange AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name,
firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM firstARIInfo
INNER JOIN lastARIInfo USING (person_id)
)
,


-- all SARS-CoV2 NAA Test Results
resultsSARSCoV2Tests AS
(
  SELECT meas.person_id, visit_occurrence_id, measurement_DATE, measurement_DATETIME, value_source_value, value_source_value IN ('Detected','Pos','Positive') AS detectedSARSCoV2, visit_concept_id, care_site_name
  FROM `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.measurement` as meas
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE measurement_concept_id = """ + str(sarscov2test_concept_id) + """
  AND meas.value_source_value is NOT NULL -- Only capture usable results and not extra descriptors
) ,

-- first SARS-CoV2 NAA Test
firstTestInfo AS
(
SELECT person_id, firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS firstSARSCoV2TestedDateTime,
    visit_concept_id AS firstTest_visit_concept_id,
    care_site_name AS firstTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME ASC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

-- last SARS-CoV2 NAA Test
lastTestInfo AS
(
SELECT person_id, lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS lastSARSCoV2TestedDateTime,
    visit_concept_id AS lastTest_visit_concept_id,
    care_site_name AS lastTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME DESC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

-- Accounting for multiple possible tests per patient, Inner join patient's first and last SARS coV2 tests
resultsSARSCoV2DateTimeRange AS(
  SELECT 
    person_id, 
    firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name,
    lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name,
    firstSARSCoV2DetectedDateTime, lastSARSCoV2DetectedDateTime
  FROM firstTestInfo
  INNER JOIN lastTestInfo USING (person_id)
  
  -- Left outer join, because not everyone who had a test will have a positive test result to join to
  
  LEFT JOIN 
  (
    SELECT person_id, MIN(measurement_DATETIME) AS firstSARSCoV2DetectedDateTime, MAX(measurement_DATETIME) AS lastSARSCoV2DetectedDateTime
    FROM resultsSARSCoV2Tests
    WHERE resultsSARSCoV2Tests.detectedSARSCoV2
    GROUP BY person_id 
  ) AS firstLastDetectedSARSCoV2Tests
  USING (person_id)
  ),

-- at this point:
--  recentARIDateTimeRange has one row per patient (41,341 rows as of 2020-04-20 cdm_release_DATE)
--  resultsSARSCoV2DateTimeRange has one row per patient (14,838)

-- now let's combine: 
-- Each with first and last dates of occurrences of ARI (after initial "recent" cutoff date) or SARS-CoV2 test results (including timing of positive/detected result, null if no positive/detected results)

recentARIandSARSCoV2DateTimeRange AS
( -- Full outer join in both directions because some people with ARI diagnosis don't get SARS-CoV2 testing and vice versa
  SELECT *,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Tested,

      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Detected,
  FROM 
    recentARIDateTimeRange FULL JOIN 
    resultsSARSCoV2DateTimeRange USING (person_id)
  -- 4,091 inner join results (recent ARI diagnosis code AND SARS-CoV2 test results exist, but not necessarily in the correct datetime order)
  -- 52,088 full outer join results (recent ARI diagnosi code OR SARS-CoV2 test results)
)

SELECT 
    recentARIandSARSCoV2DateTimeRange.*,
    year_of_birth, DATE_DIFF('2020-02-01', DATE(birth_DATETIME), YEAR) ageAsOfFeb2020,
    person.gender_concept_id , genderConc.concept_name AS genderConcept,
    person.race_source_value, person.race_concept_id, raceConc.concept_name as raceConcept,
    person.ethnicity_source_value, person.ethnicity_concept_id, ethnicityConc.concept_name as ethnicityConcept
  FROM 
    recentARIandSARSCoV2DateTimeRange 
    JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.person` AS person USING (person_id) 
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS genderConc ON (person.gender_concept_id = genderConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS raceConc ON (person.race_concept_id = raceConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS ethnicityConc ON (person.ethnicity_concept_id = ethnicityConc.concept_id)
"""

In [91]:
query_job = client.query(query)
df = query_job.to_dataframe(progress_bar_type='tqdm')

Downloading: 100%|██████████| 76275/76275 [00:23<00:00, 3101.13rows/s]


# Sanity checks on what we pulled

In [92]:
check_null_query = """
WITH

resultsSARSCoV2Tests AS
(
  SELECT meas.person_id, visit_occurrence_id, measurement_DATE, measurement_DATETIME, value_source_value, value_source_value IN ('Detected','Pos','Positive') AS detectedSARSCoV2, visit_concept_id, care_site_name
  FROM `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.measurement` as meas
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE measurement_concept_id = 706170 -- SARS-CoV2 NAA Test Result
  AND meas.value_source_value is NOT NULL -- Only capture usable results and not extra descriptors
)
  
select count(*) nRecords, count(distinct person_id) as nPerson, count(distinct visit_occurrence_id) as nVisits, countif(value_source_value is not null) as nonNullValues
from resultsSARSCoV2Tests 
"""


tmp_null_df = client.query(check_null_query).to_dataframe(progress_bar_type='tqdm')
print("Sanity check on null SARS-CoV2 NAA records:")

display(HTML(tmp_null_df.to_html()))

print("% non-null values among pulled records", 
      100.* tmp_null_df["nonNullValues"][0]/tmp_null_df["nRecords"][0])



Downloading: 100%|██████████| 1/1 [00:00<00:00,  5.87rows/s]

Sanity check on null SARS-CoV2 NAA records:


,nRecords,nPerson,nVisits,nonNullValues
0,68018,46407,65023,68018


% non-null values among pulled records 100.0


In [93]:
print('confirm person_id is unique')
df['person_id'].is_unique

confirm person_id is unique


True

In [94]:
print("Num nulls per column")
df.isnull().sum()

Num nulls per column


person_id                                  0
lastARIDateTime                        37274
lastARI_visit_concept_id               38403
lastARI_care_site_name                 45159
firstARIDateTime                       37274
firstARI_visit_concept_id              38510
firstARI_care_site_name                45608
firstSARSCoV2TestedDateTime            29868
firstTest_visit_concept_id             30145
firstTest_care_site_name               36209
lastSARSCoV2TestedDateTime             29868
lastTest_visit_concept_id              30120
lastTest_care_site_name                36613
firstSARSCoV2DetectedDateTime          74543
lastSARSCoV2DetectedDateTime           74543
daysFirstARItoFirstSARSCoV2Tested      67142
daysLastARItoFirstSARSCoV2Tested       67142
daysFirstARItoLastSARSCoV2Tested       67142
daysLastARItoLastSARSCoV2Tested        67142
daysFirstARItoFirstSARSCoV2Detected    75485
daysLastARItoFirstSARSCoV2Detected     75485
daysFirstARItoLastSARSCoV2Detected     75485
daysLastAR

# Create output columns:
1. ARI visit
2. SARSCoV2 Tested
3. SARSCoV2 Detected
4. ARI visit within a close time range of SARSCoV2 Test?
5. ARI visit within a close time range of SARSCoV2 Detection?

In [95]:
ARI_key = "ARIVisitIn" + "_".join([ARI_start_date, run_date])
ARI_description = "Patient had a visit with ARI diagnosis in " + "_".join([ARI_start_date, run_date])
ARI = df['lastARIDateTime'].notnull().rename(ARI_key)
metadata[ARI_key] = ARI_description

tested_key = "SARSCoV2TestedIn" + "_".join([ARI_start_date, run_date])
tested_description = "Patient had a SARS-CoV2 Test in " + str([ARI_start_date, run_date])
tested = df['lastSARSCoV2TestedDateTime'].notnull().rename(tested_key)
metadata[tested_key] = tested_description


detected_key = "SARSCoV2DetectedIn" + "_".join([ARI_start_date, run_date])
detected_description = "Patient had SARS-CoV2 Detected in " + "_".join([ARI_start_date, run_date])
detected = df['lastSARSCoV2DetectedDateTime'].notnull().rename(detected_key)
metadata[detected_key] = detected_description


firstorLastARItoFirstSARSCoV2TestedDayRange = [-1, 14]
ARI_tested_key = "firstorLastARItoFirstSARSCoV2TestedInDayRange" + str(firstorLastARItoFirstSARSCoV2TestedDayRange)
tmp_ARI_tested_in_range = np.logical_or(df['daysFirstARItoFirstSARSCoV2Tested'].between(firstorLastARItoFirstSARSCoV2TestedDayRange[0],
                                                            firstorLastARItoFirstSARSCoV2TestedDayRange[1]),
                                        df['daysLastARItoFirstSARSCoV2Tested'].between(firstorLastARItoFirstSARSCoV2TestedDayRange[0],
                                                            firstorLastARItoFirstSARSCoV2TestedDayRange[1]))
ARI_tested = tmp_ARI_tested_in_range.astype('int32').rename(ARI_tested_key)
ARI_tested_description = "was either the first or last ARI within the range " + str(firstorLastARItoFirstSARSCoV2TestedDayRange) + " days before the first SARSCoV2 test?"
metadata[ARI_tested_key] = ARI_tested_description
metadata["firstorLastARItoFirstSARSCoV2TestedDayRange"] = firstorLastARItoFirstSARSCoV2TestedDayRange


firstorLastARItoFirstSARSCoV2DetectedDayRange = [-1, 14]

ARI_detected_key = "firstorLastARItoFirstSARSCoV2DetectedInDayRange" + str(firstorLastARItoFirstSARSCoV2DetectedDayRange)
tmp_ARI_detected_in_range = np.logical_or(df['daysFirstARItoFirstSARSCoV2Detected'].between(firstorLastARItoFirstSARSCoV2DetectedDayRange[0],
                                                            firstorLastARItoFirstSARSCoV2DetectedDayRange[1]),
                                        df['daysLastARItoFirstSARSCoV2Detected'].between(firstorLastARItoFirstSARSCoV2DetectedDayRange[0],
                                                            firstorLastARItoFirstSARSCoV2DetectedDayRange[1]))
ARI_detected = tmp_ARI_detected_in_range.astype('int32').rename(ARI_detected_key)
ARI_detected_description = "was either the first or last ARI within the range " + str(firstorLastARItoFirstSARSCoV2DetectedDayRange) + " days before the first SARSCoV2 detection?"
metadata[ARI_detected_key] = ARI_detected_description
metadata["firstorLastARItoFirstSARSCoV2DetectedDayRange"] = firstorLastARItoFirstSARSCoV2DetectedDayRange



out_keys = [ARI_key, tested_key, detected_key, ARI_tested_key, ARI_detected_key]
print("Just created all the columns of deliverables for counts, out_keys. Use this to reference columns as needed")
print("\n".join(out_keys))

Just created all the columns of deliverables for counts, out_keys. Use this to reference columns as needed
ARIVisitIn2020-01-01_2020-06-06
SARSCoV2TestedIn2020-01-01_2020-06-06
SARSCoV2DetectedIn2020-01-01_2020-06-06
firstorLastARItoFirstSARSCoV2TestedInDayRange[-1, 14]
firstorLastARItoFirstSARSCoV2DetectedInDayRange[-1, 14]


In [96]:
# age categorical
bins = [0, 5, 18, 50, 65, 75, 85, 100]
labels = ["[{:02d}, {:02d})".format(i, j) for i, j in zip(bins[:-1], bins[1:])]
age_cat = pd.cut(df['ageAsOfFeb2020'], bins, labels=labels).rename('age')

df['care_site'] = df['firstTest_care_site_name'].fillna(df['lastARI_care_site_name'])
df['visit_concept'] = df['firstTest_visit_concept_id'].fillna(df['lastARI_visit_concept_id'])

In [97]:
df_all = pd.concat([df, ARI, tested, detected, ARI_tested, ARI_detected, age_cat], axis=1)
df_all.head()

,person_id,lastARIDateTime,lastARI_visit_concept_id,lastARI_care_site_name,firstARIDateTime,firstARI_visit_concept_id,firstARI_care_site_name,firstSARSCoV2TestedDateTime,firstTest_visit_concept_id,firstTest_care_site_name,lastSARSCoV2TestedDateTime,lastTest_visit_concept_id,lastTest_care_site_name,firstSARSCoV2DetectedDateTime,lastSARSCoV2DetectedDateTime,daysFirstARItoFirstSARSCoV2Tested,daysLastARItoFirstSARSCoV2Tested,daysFirstARItoLastSARSCoV2Tested,daysLastARItoLastSARSCoV2Tested,daysFirstARItoFirstSARSCoV2Detected,daysLastARItoFirstSARSCoV2Detected,daysFirstARItoLastSARSCoV2Detected,daysLastARItoLastSARSCoV2Detected,year_of_birth,ageAsOfFeb2020,gender_concept_id,genderConcept,race_source_value,race_concept_id,raceConcept,ethnicity_source_value,ethnicity_concept_id,ethnicityConcept,care_site,visit_concept,ARIVisitIn2020-01-01_2020-06-06,SARSCoV2TestedIn2020-01-01_2020-06-06,SARSCoV2DetectedIn2020-01-01_2020-06-06,"firstorLastARItoFirstSARSCoV2TestedInDayRange[-1, 14]","firstorLastARItoFirstSARSCoV2DetectedInDayRange[-1, 14]",age
0,32552822,2020-01-23,0.0,INTERNAL MEDICINE SAMARITAN SAN JOSE,2020-01-07,0.0,INTERNAL MEDICINE SAMARITAN SAN JOSE,NaT,NaN,None,NaT,NaN,None,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1971,49,8507,MALE,Native Hawaiian or Other Pacific Islander |,8557,Native Hawaiian or Other Pacific Islander,Unknown |,0,No matching concept,INTERNAL MEDICINE SAMARITAN SAN JOSE,0.0,True,False,False,0,0,"[18, 50)"
1,30270769,2020-01-25,0.0,FAMILY MEDICINE AFP ALAMEDA,2020-01-25,0.0,FAMILY MEDICINE AFP ALAMEDA,NaT,NaN,None,NaT,NaN,None,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962,58,8507,MALE,Native Hawaiian or Other Pacific Islander |,8557,Native Hawaiian or Other Pacific Islander,Patient Refused |,0,No matching concept,FAMILY MEDICINE AFP ALAMEDA,0.0,True,False,False,0,0,"[50, 65)"
2,43715690,2020-01-23,0.0,PEDIATRICS ALAMEDA PEDIATRICS,2020-01-23,0.0,PEDIATRICS ALAMEDA PEDIATRICS,NaT,NaN,None,NaT,NaN,None,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006,14,8507,MALE,Native Hawaiian or Other Pacific Islander |,8557,Native Hawaiian or Other Pacific Islander,Unknown |,0,No matching concept,PEDIATRICS ALAMEDA PEDIATRICS,0.0,True,False,False,0,0,"[05, 18)"
3,30539784,2020-01-03,0.0,FAMILY MEDICINE BVMG DANVILLE,2020-01-03,0.0,FAMILY MEDICINE BVMG DANVILLE,NaT,NaN,None,NaT,NaN,None,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981,39,8507,MALE,Native Hawaiian or Other Pacific Islander |,8557,Native Hawaiian or Other Pacific Islander,Patient Refused |,0,No matching concept,FAMILY MEDICINE BVMG DANVILLE,0.0,True,False,False,0,0,"[18, 50)"
4,31343658,2020-05-09,0.0,FAMILY MEDICINE OAKLAND,2020-05-09,0.0,FAMILY MEDICINE OAKLAND,2020-05-10 10:03:00,0.0,FAMILY MEDICINE OAKLAND,2020-05-10 10:03:00,0.0,FAMILY MEDICINE OAKLAND,NaT,NaT,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1969,51,8507,MALE,Native Hawaiian or Other Pacific Islander |,8557,Native Hawaiian or Other Pacific Islander,Unknown |,0,No matching concept,FAMILY MEDICINE OAKLAND,0.0,True,True,False,1,0,"[50, 65)"


In [98]:
print('Totals')
for elt in out_keys:
    df_all[elt] = df_all[elt].astype('int32')
    metadata["total_" + str(elt)] = np.sum(df_all[elt])
    print(elt, "\t", np.sum(df_all[elt]))

Totals
ARIVisitIn2020-01-01_2020-06-06 	 39001
SARSCoV2TestedIn2020-01-01_2020-06-06 	 46407
SARSCoV2DetectedIn2020-01-01_2020-06-06 	 1732
firstorLastARItoFirstSARSCoV2TestedInDayRange[-1, 14] 	 6964
firstorLastARItoFirstSARSCoV2DetectedInDayRange[-1, 14] 	 600


In [100]:

from IPython.display import display, HTML
def df_disp(df):
    display(HTML(df.to_html()))

def format_by_demo_col(demo_col, df, out_keys):
    """Get long table of values grouped by the demographic column
    """
    tbl = pd.pivot_table(df, values=out_keys, columns=[demo_col], aggfunc=np.sum)
    p_tbl = tbl.T.rename(columns=str).reset_index()
    p_tbl = p_tbl.rename({'level_0' : 'type'}, axis=1).sort_values([demo_col])
    
    m_tbl = pd.melt(p_tbl, id_vars=[demo_col], value_vars=out_keys)
    return m_tbl

def low_count_mask(df, val_col, val_range_inclusive=[1,9], val_mask=5):
    """Set values within the val_range_inclusive to val_mask
    """
    
    df[val_col][np.logical_and(df[val_col] <= val_range_inclusive[1],
                     df[val_col] >= val_range_inclusive[0])] = val_mask
    
    desc = "Values for column " + val_col + " within the inclusive range " + str(val_range_inclusive) + " were masked to " + str(val_mask) 
    
    print(desc)
    
    return df, desc


In [101]:
care_site_tok = 'care_site'
print("Saving care site counts")

def save_care_site_csv(s, tok):
    out_file = 'care_site_counts_' + tok + '_visit.csv'
    s.value_counts().to_csv(out_file)
    print("Written to ", out_file)

save_care_site_csv(df_all[care_site_tok], 'all')
for out_key in out_keys:
    save_care_site_csv(df_all[care_site_tok][df_all[out_key] == 1], out_key)

Saving care site counts
Written to  care_site_counts_all_visit.csv
Written to  care_site_counts_ARIVisitIn2020-01-01_2020-06-06_visit.csv
Written to  care_site_counts_SARSCoV2TestedIn2020-01-01_2020-06-06_visit.csv
Written to  care_site_counts_SARSCoV2DetectedIn2020-01-01_2020-06-06_visit.csv
Written to  care_site_counts_firstorLastARItoFirstSARSCoV2TestedInDayRange[-1, 14]_visit.csv
Written to  care_site_counts_firstorLastARItoFirstSARSCoV2DetectedInDayRange[-1, 14]_visit.csv


In [102]:
# change this to true if you reall ywant

export_files = []

out_folder = os.path.join("stanford_starr-omop_covid_mmwr", run_date)
os.makedirs(out_folder, exist_ok=True)
print("Writing to ", out_folder)


out_file = os.path.join(out_folder, "df_all_" + run_date + ".csv")
df_all.to_csv(out_file, index=False)
print("Writing for long table to:", out_file)
print("Not including this in the push")


val_range_inclusive = [1, 9]
metadata["val_range_inclusive"] = val_range_inclusive
print("val_range_inclusive:", val_range_inclusive)

val_mask = 5
metadata["val_mask"] = val_mask
print("val_mask:", val_mask)

demo_cols = ['age', 'ethnicityConcept', 'raceConcept', 'genderConcept']

for demo_col in demo_cols:

    out_file = os.path.join(out_folder, demo_col +"_by_demo_" + run_date + ".csv")
    df_all_by_demo = format_by_demo_col(demo_col=demo_col, df=df_all, out_keys=out_keys)
    
    df_all_by_demo, desc = low_count_mask(df=df_all_by_demo, val_col="value", 
                                    val_range_inclusive=val_range_inclusive, val_mask=val_mask)
    
    metadata[demo_col + " masking"] = desc
    
    df_disp(df_all_by_demo)
    
    df_all_by_demo.to_csv(out_file, index=False)
    
    print(out_file, "Written")
    export_files.append(out_file)


Writing to  stanford_starr-omop_covid_mmwr/2020-06-06
Writing for long table to: stanford_starr-omop_covid_mmwr/2020-06-06/df_all_2020-06-06.csv
Not including this in the push
val_range_inclusive: [1, 9]
val_mask: 5
Values for column value within the inclusive range [1, 9] were masked to 5


/share/sw/open/anaconda/3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,age,variable,value
0,"[00, 05)",ARIVisitIn2020-01-01_2020-06-06,6405
1,"[05, 18)",ARIVisitIn2020-01-01_2020-06-06,8328
2,"[18, 50)",ARIVisitIn2020-01-01_2020-06-06,13004
3,"[50, 65)",ARIVisitIn2020-01-01_2020-06-06,5997
4,"[65, 75)",ARIVisitIn2020-01-01_2020-06-06,3106
5,"[75, 85)",ARIVisitIn2020-01-01_2020-06-06,1646
6,"[85, 100)",ARIVisitIn2020-01-01_2020-06-06,422
7,"[00, 05)",SARSCoV2TestedIn2020-01-01_2020-06-06,1137
8,"[05, 18)",SARSCoV2TestedIn2020-01-01_2020-06-06,1827
9,"[18, 50)",SARSCoV2TestedIn2020-01-01_2020-06-06,24147


stanford_starr-omop_covid_mmwr/2020-06-06/age_by_demo_2020-06-06.csv Written
Values for column value within the inclusive range [1, 9] were masked to 5


,ethnicityConcept,variable,value
0,Hispanic or Latino,ARIVisitIn2020-01-01_2020-06-06,5894
1,No matching concept,ARIVisitIn2020-01-01_2020-06-06,7091
2,Not Hispanic or Latino,ARIVisitIn2020-01-01_2020-06-06,26016
3,Hispanic or Latino,SARSCoV2TestedIn2020-01-01_2020-06-06,5636
4,No matching concept,SARSCoV2TestedIn2020-01-01_2020-06-06,13894
5,Not Hispanic or Latino,SARSCoV2TestedIn2020-01-01_2020-06-06,26877
6,Hispanic or Latino,SARSCoV2DetectedIn2020-01-01_2020-06-06,355
7,No matching concept,SARSCoV2DetectedIn2020-01-01_2020-06-06,860
8,Not Hispanic or Latino,SARSCoV2DetectedIn2020-01-01_2020-06-06,517
9,Hispanic or Latino,firstorLastARItoFirstSARSCoV2TestedInDayRange[...,1049


stanford_starr-omop_covid_mmwr/2020-06-06/ethnicityConcept_by_demo_2020-06-06.csv Written
Values for column value within the inclusive range [1, 9] were masked to 5


,raceConcept,variable,value
0,American Indian or Alaska Native,ARIVisitIn2020-01-01_2020-06-06,98
1,Asian,ARIVisitIn2020-01-01_2020-06-06,6928
2,Black or African American,ARIVisitIn2020-01-01_2020-06-06,1390
3,Native Hawaiian or Other Pacific Islander,ARIVisitIn2020-01-01_2020-06-06,451
4,No matching concept,ARIVisitIn2020-01-01_2020-06-06,13089
5,White,ARIVisitIn2020-01-01_2020-06-06,17045
6,American Indian or Alaska Native,SARSCoV2TestedIn2020-01-01_2020-06-06,122
7,Asian,SARSCoV2TestedIn2020-01-01_2020-06-06,6427
8,Black or African American,SARSCoV2TestedIn2020-01-01_2020-06-06,1380
9,Native Hawaiian or Other Pacific Islander,SARSCoV2TestedIn2020-01-01_2020-06-06,472


stanford_starr-omop_covid_mmwr/2020-06-06/raceConcept_by_demo_2020-06-06.csv Written
Values for column value within the inclusive range [1, 9] were masked to 5


,genderConcept,variable,value
0,FEMALE,ARIVisitIn2020-01-01_2020-06-06,21166
1,MALE,ARIVisitIn2020-01-01_2020-06-06,17826
2,No matching concept,ARIVisitIn2020-01-01_2020-06-06,5
3,FEMALE,SARSCoV2TestedIn2020-01-01_2020-06-06,28709
4,MALE,SARSCoV2TestedIn2020-01-01_2020-06-06,17666
5,No matching concept,SARSCoV2TestedIn2020-01-01_2020-06-06,32
6,FEMALE,SARSCoV2DetectedIn2020-01-01_2020-06-06,811
7,MALE,SARSCoV2DetectedIn2020-01-01_2020-06-06,921
8,No matching concept,SARSCoV2DetectedIn2020-01-01_2020-06-06,0
9,FEMALE,firstorLastARItoFirstSARSCoV2TestedInDayRange[...,3921


stanford_starr-omop_covid_mmwr/2020-06-06/genderConcept_by_demo_2020-06-06.csv Written


In [103]:
# add this new notebook to update main repo to track changes
export_files.append("new_tbl_adj.ipynb")

# add a freeze of the notebook that was used to generate this
nb_out_file = os.path.join(out_folder, "new_tbl_adj_" + run_date + ".ipynb")
shutil.copy("new_tbl_adj.ipynb", nb_out_file)

print("Written to:", nb_out_file)
export_files.append(nb_out_file)

metadata["Freeze of exact code used to run"] =  metadata["Code repository"] + "/" + nb_out_file

meta_out_file = os.path.join(out_folder, "METADATA_" + run_date + ".CSV")

metadata["Metadata location"] = meta_out_file

metadata_df = pd.DataFrame()
metadata_df["Name"] = list(metadata.keys())
metadata_df["Description"] = list(metadata.values())
metadata_df.to_csv(meta_out_file, index=False)

print("Written to:", meta_out_file)
export_files.append(meta_out_file)




Written to: stanford_starr-omop_covid_mmwr/2020-06-06/new_tbl_adj_2020-06-06.ipynb
Written to: stanford_starr-omop_covid_mmwr/2020-06-06/METADATA_2020-06-06.CSV


In [104]:
readme_md_string = """
This repository contains long tables of: {out_key_string} grouped by demographic factors, separately: {demo_cols}

drawing from all patients in the Starr-OMOP database from {ARI_start_date} through {run_date}, inclusive.

Metadata at {metadata_loc}

Original code used to generate at {code_freeze_loc}

# CAVEATS

* For each patient, their dates and times are individually jittered by an unknown day range within [-31, 31] days. 

** Thus, there are patients appearing before the start of SARS CoV2 Testing, and patients appearing into the future.

* Note that there is a great degree of missingness in the race and ethnicity fields.

* we attempted to associate ARI visits with SARS-COV2 tests and detections by limiting to flagging patients who had a SARS-COV2 test/detection within [-1, 14] days after either their first or last ARI visit in the time range.

* For more info, see https://med.stanford.edu/starr-omop/access.html under "OMOP and STARR-OMOP documentation"

""".format(out_key_string="\n".join(out_keys),
          demo_cols=demo_cols,
           ARI_start_date=ARI_start_date,
           run_date=run_date,
            metadata_loc = metadata["Metadata location"],
           code_freeze_loc= metadata["Freeze of exact code used to run"])

readme_md_out_file = os.path.join(out_folder, "README.md")
with open(readme_md_out_file, 'w') as f:
    f.write(readme_md_string)

print("Written to ", readme_md_out_file)
if not readme_md_out_file in export_files:
    export_files.append(readme_md_out_file)

Written to  stanford_starr-omop_covid_mmwr/2020-06-06/README.md


In [105]:
print("Confirm the files you are planning to export below? Writing to export_list.txt")

with open("export_list.txt", 'w') as f:
    for export_file in export_files:
        f.write(export_file + "\n")
        
print("")
for export_file in export_files:
    print(export_file)

Confirm the files you are planning to export below? Writing to export_list.txt

stanford_starr-omop_covid_mmwr/2020-06-06/age_by_demo_2020-06-06.csv
stanford_starr-omop_covid_mmwr/2020-06-06/ethnicityConcept_by_demo_2020-06-06.csv
stanford_starr-omop_covid_mmwr/2020-06-06/raceConcept_by_demo_2020-06-06.csv
stanford_starr-omop_covid_mmwr/2020-06-06/genderConcept_by_demo_2020-06-06.csv
new_tbl_adj.ipynb
stanford_starr-omop_covid_mmwr/2020-06-06/new_tbl_adj_2020-06-06.ipynb
stanford_starr-omop_covid_mmwr/2020-06-06/METADATA_2020-06-06.CSV
stanford_starr-omop_covid_mmwr/2020-06-06/README.md


In [106]:
print("Now that export_list is updated, open a terminal and do:")
print('cd "/home/jhlu/CDSS/scripts/Covid19 CDC/"')
print('./git_export.sh')

Now that export_list is updated, open a terminal and do:
cd "/home/jhlu/CDSS/scripts/Covid19 CDC/"
./git_export.sh


# BELOW IS OPTIONAL SECONDARY ANALYSIS, NOT NECESSARY TO RUN

In [ ]:
df_all['ethnicity_source_value'].value_counts().to_csv('ethnicity_source')
df_all['race_source_value'].value_counts().to_csv('race_source')
# for ind, grp in df_all.groupby('raceConcept'):
#     print(ind)
#     print(grp['race_source_value'].unique())
for ind, grp in df_all.groupby('ethnicityConcept'):
    print(ind)
    print(grp['ethnicity_source_value'].unique())

In [56]:
def format_by_visit(ind_nm, df, out_keys, visit_token = 'visit_concept'):
    tbl = pd.pivot_table(df, values=out_keys, index=[ind_nm],
                            columns=[visit_token], aggfunc=np.sum)
    p_tbl = tbl.T.rename(columns=str).reset_index()
    p_tbl = p_tbl.rename({'level_0' : 'type'}, axis=1).sort_values([visit_token, 'type'])

    cols = p_tbl.columns.tolist()
    cols = [cols[1], cols[0]] + cols[2:]
    p_tbl = p_tbl[cols]
    df_disp(p_tbl)
    return p_tbl

def format_by_ind(ind_nm, df, out_keys):
    tbl = pd.pivot_table(df, values=out_keys, index=[ind_nm],
                            aggfunc=np.sum)
    p_tbl = tbl.T
    
    df_disp(p_tbl)
    return p_tbl

In [ ]:
# get wide tables
for elt in ['age', 'ethnicityConcept', 'raceConcept', 'genderConcept']:
    format_by_visit(ind_nm=elt, df=df_all, out_keys=out_keys)
    format_by_ind(ind_nm=elt, df=df_all, out_keys=out_keys)

In [ ]:
print("Oldest nSars-COV2-test")
print(print(min(df_all[df_all["firstSARSCoV2TestedDateTime"].notnull()]["firstSARSCoV2TestedDateTime"])))
print("Newest nSars-COV2-test")
print(print(max(df_all[df_all["lastSARSCoV2TestedDateTime"].notnull()]["lastSARSCoV2TestedDateTime"])))
print("---")
print("Oldest nSars-COV2-detection")
print(min(df_all[df_all["firstSARSCoV2DetectedDateTime"].notnull()]["firstSARSCoV2DetectedDateTime"]))
print("Newest nSars-COV2-detection")
print(max(df_all[df_all["lastSARSCoV2DetectedDateTime"].notnull()]["lastSARSCoV2DetectedDateTime"]))
print("---")

plt.figure()
plt.hist(df_all[df_all["firstSARSCoV2TestedDateTime"].notnull()]["firstSARSCoV2TestedDateTime"],
        alpha=0.5, label="First SARSCoV2Test")
plt.hist(df_all[df_all["lastSARSCoV2TestedDateTime"].notnull()]["lastSARSCoV2TestedDateTime"],
        alpha=0.5, label="Last SARSCoV2Test")
plt.title("First and Last SARS-COV2 test dates as of \n" + datetime.datetime.today().strftime("%m/%d/%Y"))
plt.legend()
plt.show()
plt.close()

plt.figure()
plt.hist(df_all[df_all["firstSARSCoV2DetectedDateTime"].notnull()]["firstSARSCoV2DetectedDateTime"],
        alpha=0.5, label="First SARSCoV2Detected")
plt.hist(df_all[df_all["lastSARSCoV2DetectedDateTime"].notnull()]["lastSARSCoV2DetectedDateTime"],
        alpha=0.5, label="Last SARSCoV2Detected")
plt.title("First and Last SARS-COV2 detection dates as of\n" + datetime.datetime.today().strftime("%m/%d/%Y"))
plt.xticks(rotation=90)
plt.legend()
plt.show()
plt.close()
